In [12]:
import sys
sys.path.insert(1, './src')
from crfrnn_model import get_crfrnn_model_def
import util
from skimage.io import imread
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, ZeroPadding2D, \
    Dropout, Conv2DTranspose, Cropping2D, Add, UpSampling2D
from keras.layers.merge import concatenate
from keras_segmentation.models.model_utils import get_segmentation_model

In [13]:
import tensorflow as tf
from tensorflow import keras

In [14]:
from keras.preprocessing.image import ImageDataGenerator
import glob
import skimage.io as io
import skimage.transform as trans
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from keras import Model

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [19]:
from crfrnn_layer import CrfRnnLayer
from keras.models import Model
from keras_segmentation.models.model_utils import get_segmentation_model

""" Returns Keras CRN-RNN model definition.

Currently, only 500 x 500 images are supported. However, one can get this to
work with different image sizes by adjusting the parameters of the Cropping2D layers
below.
"""

channels, height, width = 3, 500, 500

n_classes = 2 #21

# Input
input_shape = (height, width, 3)
img_input = Input(shape=input_shape)

# Add plenty of zero padding
x = ZeroPadding2D(padding=(100, 100))(img_input)

# VGG-16 convolution block 1
x = Conv2D(64, (3, 3), activation='relu', padding='valid', name='conv1_1')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='conv1_2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='pool1')(x)

# VGG-16 convolution block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='conv2_1')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='conv2_2')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='pool2', padding='same')(x)

# VGG-16 convolution block 3
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_1')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_2')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='pool3', padding='same')(x)
pool3 = x

# VGG-16 convolution block 4
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='pool4', padding='same')(x)
pool4 = x

# VGG-16 convolution block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_3')(x)
x = MaxPooling2D((2, 2), strides=(2, 2), name='pool5', padding='same')(x)

# Fully-connected layers converted to convolution layers
x = Conv2D(4096, (7, 7), activation='relu', padding='valid', name='fc6')(x)
x = Dropout(0.5)(x)
x = Conv2D(4096, (1, 1), activation='relu', padding='valid', name='fc7')(x)
x = Dropout(0.5)(x)
x = Conv2D(n_classes, (1, 1), padding='valid', name='score-fr')(x)

# Deconvolution
score2 = Conv2DTranspose(n_classes, (4, 4), strides=2, name='score2')(x)

# Skip connections from pool4
score_pool4 = Conv2D(n_classes, (1, 1), name='score-pool4')(pool4)
score_pool4c = Cropping2D((5, 5))(score_pool4)
score_fused = Add()([score2, score_pool4c])
score4 = Conv2DTranspose(n_classes, (4, 4), strides=2, name='score4', use_bias=False)(score_fused)

# Skip connections from pool3
score_pool3 = Conv2D(n_classes, (1, 1), name='score-pool3')(pool3)
score_pool3c = Cropping2D((9, 9))(score_pool3)

# Fuse things together
score_final = Add()([score4, score_pool3c])

# Final up-sampling and cropping
upsample = Conv2DTranspose(n_classes, (16, 16), strides=8, name='upsample', use_bias=False)(score_final)
upscore = Cropping2D(((31, 37), (31, 37)))(upsample)

output = CrfRnnLayer(image_dims=(height, width),
                     num_classes=n_classes,
                     theta_alpha=160.,
                     theta_beta=3.,
                     theta_gamma=3.,
                     num_iterations=10,
                     name='crfrnn')([upscore, img_input])

# Build the model
#model_crfrnn = Model(img_input, output)
model_crfrnn = get_segmentation_model(img_input, output)

In [22]:
model_crfrnn.train(train_images = "/Users/mavaylon/Research/Pytorch_UNet/Data/Talita/Talita_500x500/orig/", 
            train_annotations="/Users/mavaylon/Research/Pytorch_UNet/Data/Talita/Talita_500x500/gt/",
                   #checkpointpath?
            epochs=5)

  0%|          | 0/512 [00:00<?, ?it/s]

Verifying training dataset
The pixel values of the segmentation image /Users/mavaylon/Research/Pytorch_UNet/Data/Talita/Talita_500x500/gt/image_408.png violating range [0, 1]. Found maximum pixel value 255
Dataset not verified!


AssertionError: 